# Rare Variant Association Testing Tutorial

This tutorial demonstrates how to perform rare variant association testing using burden tests and SKAT (Sequence Kernel Association Test). We will walk through the steps of preparing the single-cell data (the same as for [common variant eQTL analysis](./pseudobulk_eqtl.ipynb)), running burden tests, and combining p-values across annotations. Additionally, we will explore SKAT for association testing.

The tutorial exemplifies the rare variant associationg testing procedure for a single cell type (CD 8 NC).

To get the VEP output files used in this tutorial check the [variant annotation tutorial](./explore_annotations.ipynb)

## Objectives
- Learn how to prepare genotype and phenotype data for rare variant association testing.
- Understand how to apply burden tests with different weighting schemes.
- Combine p-values across annotations using the Cauchy combination test.
- Perform SKAT for rare variant association testing.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [38]:
import gc
from pathlib import Path
import warnings

import anndata as ad
import scanpy as sc
import dask.array as da
import numpy as np
from tqdm.auto import tqdm

import cellink as cl
from cellink._core import DAnn, GAnn
from cellink.tl._rvat import run_burden_test, run_skat_test, beta_weighting
from cellink.utils import column_normalize, gaussianize
from cellink.at.acat import acat_test

In [37]:
DATA = Path(cl.__file__).parent.parent.parent / "data"
DATA = Path("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/input_data")
GENODATA = DATA  # / "OneK1K_imputation_post_qc_r2_08"

gpc_path = GENODATA / "pcdir/wgs.dose.filtered.R2_0.8.filtered.pruned.eigenvec"
adata_path = DATA / "OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad.gz"
gdata_path = GENODATA / "filter_vcf_r08/chr22.dose.filtered.R2_0.8.vcz"

In [59]:
DATA = Path(cl.__file__).parent.parent.parent / "data"
DATA = Path("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/input_data/eqtl_catalog")
GENODATA = DATA  # / "OneK1K_imputation_post_qc_r2_08"

# gpc_path = GENODATA / "pcdir/wgs.dose.filtered.R2_0.8.filtered.pruned.eigenvec"
adata_path = DATA / "onk1k_cellxgene.h5ad"
gdata_path = GENODATA / "sample.vcz"

In [39]:
adata = ad.read_h5ad(adata_path)
adata

AnnData object with n_obs × n_vars = 1272489 × 32738
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'pool', 'individual', 'percent.mt', 'latent', 'nCount_SCT', 'nFeature_SCT', 'cell_type', 'cell_label', 'sex', 'age'
    var: 'GeneSymbol', 'features'

In [60]:
gdata = cl.io.read_sgkit_zarr(gdata_path)

In [63]:
gdata.obs

""
donor_id
OneK1K_1
OneK1K_10
OneK1K_100
OneK1K_1000
OneK1K_1001
...
OneK1K_995
OneK1K_996
OneK1K_997


In [48]:
adata.obs[["cell_type", "predicted.celltype.l2.score", "predicted.celltype.l2"]]

,cell_type,predicted.celltype.l2.score,predicted.celltype.l2
barcode,,,
AAACCTGAGAATGTTG-1,"central memory CD4-positive, alpha-beta T cell",0.605378,CD4 TCM
AAACCTGAGAGAACAG-1,natural killer cell,1.000000,NK
AAACCTGAGCATGGCA-1,"naive thymus-derived CD4-positive, alpha-beta ...",0.557355,CD4 Naive
AAACCTGAGTATTGGA-1,"effector memory CD8-positive, alpha-beta T cell",0.359614,CD8 TEM
AAACCTGAGTGTCCCG-1,"effector memory CD8-positive, alpha-beta T cell",0.896674,CD8 TEM
...,...,...,...
TTTGTCATCCGCTGTT-9,transitional stage B cell,0.811524,B intermediate
TTTGTCATCCGTTGTC-9,"central memory CD4-positive, alpha-beta T cell",0.922046,CD4 TCM
TTTGTCATCGCCGTGA-9,"naive thymus-derived CD4-positive, alpha-beta ...",0.763648,CD4 Naive


In [64]:
adata.obs["donor_id"]

barcode
AAACCTGAGAATGTTG-1    691_692
AAACCTGAGAGAACAG-1    693_694
AAACCTGAGCATGGCA-1    688_689
AAACCTGAGTATTGGA-1    683_684
AAACCTGAGTGTCCCG-1    684_685
                       ...   
TTTGTCATCCGCTGTT-9    796_797
TTTGTCATCCGTTGTC-9    800_801
TTTGTCATCGCCGTGA-9    821_822
TTTGTCATCGCGGATC-9    840_841
TTTGTCATCTCGTATT-9    821_822
Name: donor_id, Length: 1248980, dtype: category
Categories (981, object): ['1_1', '2_2', '3_3', '4_4', ..., '1078_1079', '1079_1080', '1080_1081', '1081_1082']

In [52]:
adata.obs["donor_id"]

barcode
AAACCTGAGAATGTTG-1    691_692
AAACCTGAGAGAACAG-1    693_694
AAACCTGAGCATGGCA-1    688_689
AAACCTGAGTATTGGA-1    683_684
AAACCTGAGTGTCCCG-1    684_685
                       ...   
TTTGTCATCCGCTGTT-9    796_797
TTTGTCATCCGTTGTC-9    800_801
TTTGTCATCGCCGTGA-9    821_822
TTTGTCATCGCGGATC-9    840_841
TTTGTCATCTCGTATT-9    821_822
Name: donor_id, Length: 1248980, dtype: category
Categories (981, object): ['1_1', '2_2', '3_3', '4_4', ..., '1078_1079', '1079_1080', '1080_1081', '1081_1082']

In [54]:
len(adata.obs["cell_type"].value_counts())

29

In [55]:
len(adata.obs["predicted.celltype.l2"].value_counts())

31

In [42]:
adata = adata2

In [36]:
10_000_000 / 42

238095.2380952381

In [43]:
n_gpcs = 20
n_epcs = 15
batch_e_pcs_n_top_genes = 2000
chrom = 22
cis_window = 100_000
cell_type = "CD8 NC"
pb_gex_key = f"PB_{cell_type}"  # pseudobulk expression in dd.G.obsm[key_added]
original_donor_col = "individual"
min_percent_donors_expressed = 0.1
celltype_key = "cell_label"
do_debug = False

## Prepare data 

In [6]:
if do_debug:
    adata_path = DATA / "debug_OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad"

adata = ad.read_h5ad(adata_path)
gdata = cl.io.read_sgkit_zarr(gdata_path)

gene_ann = pd.read_csv(DATA / "gene_annotation.csv").set_index("ensembl_gene_id")
adata.var = pd.concat([adata.var, gene_ann.loc[adata.var.index]], axis=1).rename(
    columns={
        "start_position": GAnn.start,
        "end_position": GAnn.end,
        "chromosome_name": GAnn.chrom,
    }
)
adata.obs[DAnn.donor] = adata.obs[original_donor_col]
adata

AnnData object with n_obs × n_vars = 1272489 × 32738
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'pool', 'individual', 'percent.mt', 'latent', 'nCount_SCT', 'nFeature_SCT', 'cell_type', 'cell_label', 'sex', 'age', 'donor_id'
    var: 'GeneSymbol', 'features', 'start', 'end', 'chrom', 'strand', 'description', 'wikigene_name', 'wikigene_id'

In [7]:
dd = cl.DonorData(G=gdata, C=adata).copy()  # copy to avoid view warnings
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │       DonorData(981 x n_donors=143,083, n_cells_per_donor=[343-3,523], donor_id = 'donor_id')    │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 143,083 │ AnnData object with n_obs × n_vars = 1,272,489 ×   │
│                                                    │ 32,738                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2',  │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'  │ 'nFeature_RNA', 'pool', 'individual',              │
│                                                    │ 'percent.mt', 'latent', 'nCount_SCT',              │
│                                                    │ 'nFeature_SCT', 'cell_type', 'cell_label', 'sex',  │
│                                                    │ 'age', 'donor_id'                                  │
│     varm: 'filter'                                 │     var: 'GeneSymbol', 'features', 'start', 'end', │
│                                                    │ 'chrom', 'strand', 'description', 'wikigene_name', │
│                                                    │ 'wikigene_id'                                      │
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────┘

### Preprocessing Single-Cell Data
We normalize and log-transform the expression data, compute highly variable genes, and perform PCA to extract expression principal components (ePCs). 

In [8]:
sc.pp.normalize_total(dd.C)
sc.pp.log1p(dd.C)
sc.pp.normalize_total(dd.C)

# are the expression pcs computed by pseudobulking across all cell types?
mdata = sc.get.aggregate(dd.C, by=DAnn.donor, func="mean")
mdata.X = mdata.layers.pop("mean")

sc.pp.highly_variable_genes(mdata, n_top_genes=batch_e_pcs_n_top_genes)
sc.tl.pca(mdata, n_comps=n_epcs)

dd.G.obsm["ePCs"] = mdata[dd.G.obs_names].obsm["X_pca"]

In [9]:
dd = dd[..., dd.C.obs[celltype_key] == cell_type, :].copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(981 x n_donors=143,083, n_cells_per_donor=[9-479], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 143,083 │ AnnData object with n_obs × n_vars = 133,482 ×     │
│                                                    │ 32,738                                             │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2',  │     obs: 'orig.ident', 'nCount_RNA',               │
│ 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'  │ 'nFeature_RNA', 'pool', 'individual',              │
│                                                    │ 'percent.mt', 'latent', 'nCount_SCT',              │
│                                                    │ 'nFeature_SCT', 'cell_type', 'cell_label', 'sex',  │
│                                                    │ 'age', 'donor_id'                                  │
│     obsm: 'ePCs'                                   │     var: 'GeneSymbol', 'features', 'start', 'end', │
│                                                    │ 'chrom', 'strand', 'description', 'wikigene_name', │
│                                                    │ 'wikigene_id'                                      │
│     varm: 'filter'                                 │     uns: 'log1p'                                   │
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────┘

In [10]:
gc.collect()

999

In [11]:
dd.aggregate(key_added=pb_gex_key, sync_var=True, verbose=True)
dd.aggregate(obs=["sex", "age"], func="first", add_to_obs=True)
dd

[2025-04-22 16:36:00,837] INFO:cellink._core.donordata: Aggregated X to PB_CD8 NC
[2025-04-22 16:36:00,838] INFO:cellink._core.donordata: Observation found for 981 donors.


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(981 x n_donors=143,083, n_cells_per_donor=[9-479], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 143,083 │ AnnData object with n_obs × n_vars = 133,482 ×     │
│                                                    │ 32,738                                             │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'pool', 'individual',              │
│                                                    │ 'percent.mt', 'latent', 'nCount_SCT',              │
│                                                    │ 'nFeature_SCT', 'cell_type', 'cell_label', 'sex',  │
│                                                    │ 'age', 'donor_id'                                  │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2',  │     var: 'GeneSymbol', 'features', 'start', 'end', │
│ 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'  │ 'chrom', 'strand', 'description', 'wikigene_name', │
│                                                    │ 'wikigene_id'                                      │
│     obsm: 'ePCs', 'PB_CD8 NC'                      │     uns: 'log1p'                                   │
│     varm: 'filter'                                 │                                                    │
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────┘

In [12]:
dd.G.obsm[pb_gex_key].shape

(981, 32738)

In [13]:
gpcs = pd.read_csv(gpc_path, sep=r"\s+", index_col=1, header=None).drop(columns=0)
dd.G.obsm["gPCs"] = gpcs.loc[dd.G.obs_names].iloc[:, :n_gpcs]

In [14]:
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

keep_genes = ((dd.G.obsm[pb_gex_key] > 0).mean(axis=0) >= min_percent_donors_expressed).values
dd = dd[..., keep_genes]
print("after filtering")
print(f"{pb_gex_key} shape:", dd.G.obsm[pb_gex_key].shape)
print("dd.shape:", dd.shape)

PB_CD8 NC shape: (981, 32738)
dd.shape: (981, 143083, 133482, 32738)
after filtering
PB_CD8 NC shape: (981, 14119)
dd.shape: (981, 143083, 133482, 14119)


In [16]:
# alternative to dd[:, dd.G.var.chrom == str(chrom), :, dd.C.var.chrom == str(chrom)]
dd = dd.sel(G_var=dd.G.var.chrom == str(chrom), C_var=dd.C.var.chrom == str(chrom)).copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │         DonorData(981 x n_donors=143,083, n_cells_per_donor=[9-479], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 143,083 │ AnnData object with n_obs × n_vars = 133,482 × 360 │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'pool', 'individual',              │
│                                                    │ 'percent.mt', 'latent', 'nCount_SCT',              │
│                                                    │ 'nFeature_SCT', 'cell_type', 'cell_label', 'sex',  │
│                                                    │ 'age', 'donor_id'                                  │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2',  │     var: 'GeneSymbol', 'features', 'start', 'end', │
│ 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'  │ 'chrom', 'strand', 'description', 'wikigene_name', │
│                                                    │ 'wikigene_id'                                      │
│     obsm: 'ePCs', 'PB_CD8 NC', 'gPCs'              │     uns: 'log1p'                                   │
│     varm: 'filter'                                 │                                                    │
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────┘

### Adding variant annotations to `dd`
We use VEP (Variant Effect Predictor) annotations to add functional information to the variants (as explained [here](./explore_annotations.ipynb)). These annotations are aggregated and stored in the `variant_annotation` attribute of the genotype data.

In [17]:
vep_annotation_file = "/data/nasif12/home_if12/hoev/git/sc-genetics/tests/data/variants_vep_annotated_all_ch22.txt"

In [18]:
cl.tl.add_vep_annos_to_gdata(vep_anno_file=vep_annotation_file, gdata=dd.G, dummy_consequence=True)
dd.G.uns["variant_annotation_vep"]

[2025-04-22 16:36:02,739] INFO:cellink.tl._annotate_snps_genotype_data: Preparing VEP annotations for addition to gdata
[2025-04-22 16:36:02,740] INFO:cellink.tl._annotate_snps_genotype_data: Reading annotation file /data/nasif12/home_if12/hoev/git/sc-genetics/tests/data/variants_vep_annotated_all_ch22.txt
[2025-04-22 16:36:03,154] INFO:cellink.tl._annotate_snps_genotype_data: Annotation file loaded
[2025-04-22 16:36:03,179] INFO:cellink.tl._annotate_snps_genotype_data: Annotation columns: ['snp_id', 'Location', 'Allele', 'gene_id', 'transcript_id', 'Feature_type', 'Consequence', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'BIOTYPE', 'CANONICAL', 'ENSP', 'SIFT', 'PolyPhen', 'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'CADD_PHRED', 'CADD_RAW', 'TSS

Consequence_3_prime_UTR_variant  \
snp_id          gene_id         transcript_id                                      
22_16849573_A_G -               -                                              0   
22_16849971_A_T -               -                                              0   
22_16850437_G_A -               -                                              0   
22_16851225_C_T -               -                                              0   
22_16851356_C_T -               -                                              0   
...                                                                          ...   
22_51211031_A_G ENSG00000184319 ENST00000496652                                0   
22_51213613_C_T ENSG00000184319 ENST00000496652                                0   
                ENSG00000079974 ENST00000395593                                0   
22_51216564_T_C ENSG00000184319 ENST00000496652                                0   
                ENSG00000079974 ENST00000395593                                0   

                                                 Consequence_5_prime_UTR_variant  \
snp_id          gene_id         transcript_id                                      
22_16849573_A_G -               -                                              0   
22_16849971_A_T -               -                                              0   
22_16850437_G_A -               -                                              0   
22_16851225_C_T -               -                                              0   
22_16851356_C_T -               -                                              0   
...                                                                          ...   
22_51211031_A_G ENSG00000184319 ENST00000496652                                0   
22_51213613_C_T ENSG00000184319 ENST00000496652                                0   
                ENSG00000079974 ENST00000395593                                0   
22_51216564_T_C ENSG00000184319 ENST00000496652                                0   
                ENSG00000079974 ENST00000395593                                0   

                                                 Consequence_NMD_transcript_variant  \
snp_id          gene_id         transcript_id                                         
22_16849573_A_G -               -                                                 0   
22_16849971_A_T -               -                                                 0   
22_16850437_G_A -               -                                                 0   
22_16851225_C_T -               -                                                 0   
22_16851356_C_T -               -                                                 0   
...                                                                             ...   
22_51211031_A_G ENSG00000184319 ENST00000496652                                   0   
22_51213613_C_T ENSG00000184319 ENST00000496652                                   0   
                ENSG00000079974 ENST00000395593                                   0   
22_51216564_T_C ENSG00000184319 ENST00000496652                                   0   
                ENSG00000079974 ENST00000395593                                   0   

                                                 Consequence_coding_sequence_variant  \
snp_id          gene_id         transcript_id                                          
22_16849573_A_G -               -                                                  0   
22_16849971_A_T -               -                                                  0   
22_16850437_G_A -               -                                                  0   
22_16851225_C_T -               -                                                  0   
22_16851356_C_T -               -                                                  0   
...                                                                              ...   
22_51211031_A_G ENSG0000018

In [19]:
cl.tl.aggregate_annotations_for_varm(
    dd.G, "variant_annotation_vep", agg_type="first", return_data=True
)  # TODO change agg type

[2025-04-22 16:36:06,228] INFO:cellink.tl._annotate_snps_genotype_data: Aggregating using method: first


,gene_id,transcript_id,Consequence_3_prime_UTR_variant,Consequence_5_prime_UTR_variant,Consequence_NMD_transcript_variant,Consequence_coding_sequence_variant,Consequence_downstream_gene_variant,Consequence_incomplete_terminal_codon_variant,Consequence_intergenic_variant,Consequence_intron_variant,...,ENSP,BIOTYPE,SIFT,gnomADe_OTH_AF,CADD_RAW,gnomADe_ASJ_AF,DISTANCE,CADD_PHRED,STRAND,CDS_position
snp_id,,,,,,,,,,,,,,,,,,,,,
22_16849573_A_G,-,-,0,0,0,0,0,0,1,0,...,-,-,NaN,NaN,0.433139,NaN,NaN,8.747,NaN,-
22_16849971_A_T,-,-,0,0,0,0,0,0,1,0,...,-,-,NaN,NaN,0.442607,NaN,NaN,8.843,NaN,-
22_16850437_G_A,-,-,0,0,0,0,0,0,1,0,...,-,-,NaN,NaN,0.369731,NaN,NaN,8.063,NaN,-
22_16851225_C_T,-,-,0,0,0,0,0,0,1,0,...,-,-,NaN,NaN,0.393139,NaN,NaN,8.324,NaN,-
22_16851356_C_T,-,-,0,0,0,0,0,0,1,0,...,-,-,NaN,NaN,0.377289,NaN,NaN,8.148,NaN,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22_51202748_A_G,ENSG00000184319,ENST00000496652,0,0,0,0,0,0,0,1,...,-,processed_transcript,NaN,NaN,0.550962,NaN,NaN,9.870,1.0,-
22_51208568_G_T,ENSG00000184319,ENST00000496652,0,0,0,0,0,0,0,1,...,-,processed_transcript,NaN,NaN,0.102244,NaN,NaN,4.299,1.0,-
22_51211031_A_G,ENSG00000079974,ENST00000395593,0,0,0,0,0,0,0,1,...,ENSP00000378958,protein_coding,NaN,NaN,0.153297,NaN,NaN,5.127,-1.0,-


In [20]:
dd.G.varm["variant_annotation"].columns

Index(['gene_id', 'transcript_id', 'Consequence_3_prime_UTR_variant',
       'Consequence_5_prime_UTR_variant', 'Consequence_NMD_transcript_variant',
       'Consequence_coding_sequence_variant',
       'Consequence_downstream_gene_variant',
       'Consequence_incomplete_terminal_codon_variant',
       'Consequence_intergenic_variant', 'Consequence_intron_variant',
       'Consequence_mature_miRNA_variant', 'Consequence_missense_variant',
       'Consequence_non_coding_transcript_exon_variant',
       'Consequence_non_coding_transcript_variant',
       'Consequence_splice_acceptor_variant',
       'Consequence_splice_donor_5th_base_variant',
       'Consequence_splice_donor_region_variant',
       'Consequence_splice_donor_variant',
       'Consequence_splice_polypyrimidine_tract_variant',
       'Consequence_splice_region_variant', 'Consequence_start_lost',
       'Consequence_stop_gained', 'Consequence_stop_retained_variant',
       'Consequence_synonymous_variant', 'Consequence_ups

In [21]:
dd.G.uns["variant_annotation_vep"]["CADD_RAW"].describe()

count    178597.000000
mean         -0.000616
std           0.407068
min          -2.286863
25%          -0.202241
50%          -0.040239
75%           0.123338
max           7.595543
Name: CADD_RAW, dtype: float64

In [22]:
dd.G.uns["variant_annotation_vep"]["TSSDistance"].describe()

count    24466.000000
mean      2536.080724
std       1454.770799
min          1.000000
25%       1272.000000
50%       2561.000000
75%       3803.000000
max       5000.000000
Name: TSSDistance, dtype: float64

## Run the burden test

Burden tests aggregate the effects of rare variants within a gene or region to test for association with a phenotype (cell type specific expression). We use different weighting schemes, including:
- **CADD_RAW**: Raw CADD scores.
- **maf_beta**: Beta weighting based on minor allele frequency (MAF).
- **tss_distance**: Distance to the transcription start site (TSS).
- **tss_distance_exp**: Exponential weighting based on TSS distance.



In [23]:
burden_agg_fct = "sum"
run_lrt = True
annotation_cols = ["CADD_RAW", "maf_beta", "tss_distance", "tss_distance_exp"]
rare_maf_threshold = 0.01

### Filtering for Rare Variants
We filter variants with a minor allele frequency (MAF) below a specified threshold (e.g., 0.01) to focus on rare variants.

In [24]:
dd = dd.sel(G_var=dd.G.var.maf < rare_maf_threshold).copy()
dd

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮        
       │          DonorData(981 x n_donors=46,642, n_cells_per_donor=[9-479], donor_id = 'donor_id')      │        
       ╰──────────────────────────────────────────────────────────────────────────────────────────────────╯        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ G (donors)                                         ┃ C (cells)                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ AnnData object with n_obs × n_vars = 981 × 46,642  │ AnnData object with n_obs × n_vars = 133,482 × 360 │
│     obs: 'sex', 'age'                              │     obs: 'orig.ident', 'nCount_RNA',               │
│                                                    │ 'nFeature_RNA', 'pool', 'individual',              │
│                                                    │ 'percent.mt', 'latent', 'nCount_SCT',              │
│                                                    │ 'nFeature_SCT', 'cell_type', 'cell_label', 'sex',  │
│                                                    │ 'age', 'donor_id'                                  │
│     var: 'chrom', 'pos', 'a0', 'a1', 'AF', 'ER2',  │     var: 'GeneSymbol', 'features', 'start', 'end', │
│ 'maf', 'R2', 'contig', 'id', 'id_mask', 'quality'  │ 'chrom', 'strand', 'description', 'wikigene_name', │
│                                                    │ 'wikigene_id'                                      │
│     uns: 'variant_annotation_vep'                  │     uns: 'log1p'                                   │
│     obsm: 'ePCs', 'PB_CD8 NC', 'gPCs'              │                                                    │
│     varm: 'filter', 'variant_annotation'           │                                                    │
└────────────────────────────────────────────────────┴────────────────────────────────────────────────────┘

### Custom MAF Weights

We add custom MAF weights commonly used in burden tests, such as `Beta(MAF, 1, 25)`. These weights prioritize rarer variants in the analysis. TSS distance weight as used in the SAIGE-QTL paper are added manually for each gene

In [27]:
dd.G.varm["variant_annotation"]["maf_beta"] = beta_weighting(dd.G.var["maf"])

### Run burden tests using each annotation individually for weighting


In [ ]:
# This specifies covariates/fixed effects
F = np.concatenate(
    [
        np.ones((dd.shape[0], 1)),
        dd.G.obs[["sex"]].values - 1,
        dd.G.obs[["age"]].values,
        dd.G.obsm["gPCs"].values,
        dd.G.obsm["ePCs"],
    ],
    axis=1,
)
F[:, 2:] = column_normalize(F[:, 2:])

In [ ]:
results = []
if isinstance(dd.G.X, da.Array | ad._core.views.DaskArrayView):
    if dd.G.is_view:
        dd._G = dd._G.copy()
    dd.G.X = dd.G.X.compute()

if do_debug:
    warnings.filterwarnings("ignore", category=RuntimeWarning)

for gene, row in tqdm(dd.C.var.iterrows(), total=dd.shape[3]):
    Y = gaussianize(dd.G.obsm[pb_gex_key][[gene]].values + 1e-5 * np.random.randn(dd.shape[0], 1))

    start = max(0, row.start - cis_window)
    end = row.end + cis_window
    _G = dd.G[:, (dd.G.var.pos < end)]
    _G = _G[:, (_G.var.pos > start)]
    _G = _G[:, (_G.X.std(0) != 0)]
    _G = _G.copy()

    # TODO make strand aware
    _G.varm["variant_annotation"]["tss_distance"] = np.abs(row.start - _G.var["pos"])
    _G.varm["variant_annotation"]["tss_distance_exp"] = np.exp(-1e-5 * _G.varm["variant_annotation"]["tss_distance"])

    rdf = run_burden_test(
        _G, Y, F, gene, annotation_cols=annotation_cols, burden_agg_fct=burden_agg_fct, run_lrt=run_lrt
    )
    results.append(rdf)

rdf = pd.concat(results)
rdf

100%|██████████| 360/360 [00:12<00:00, 29.54it/s]


,burden_gene,egene,weight_col,burden_agg_fct,pv,beta,betaste,lrt
0,ENSG00000100181,ENSG00000100181,CADD_RAW,sum,0.289885,1.213162e+00,1.146252e+00,1.120153
1,ENSG00000100181,ENSG00000100181,maf_beta,sum,0.025325,-9.702817e-03,4.338561e-03,5.001546
2,ENSG00000100181,ENSG00000100181,tss_distance,sum,0.094869,-2.169492e-06,1.298900e-06,2.789745
3,ENSG00000100181,ENSG00000100181,tss_distance_exp,sum,0.032786,-3.156013e-01,1.478437e-01,4.556923
0,ENSG00000237438,ENSG00000237438,CADD_RAW,sum,0.164809,8.317045e-02,5.987450e-02,1.929542
...,...,...,...,...,...,...,...,...
3,ENSG00000100299,ENSG00000100299,tss_distance_exp,sum,0.811293,-4.636869e-03,1.942074e-02,0.057006
0,ENSG00000079974,ENSG00000079974,CADD_RAW,sum,0.134451,-3.387797e-01,2.263393e-01,2.240345
1,ENSG00000079974,ENSG00000079974,maf_beta,sum,0.297102,-2.105266e-03,2.019116e-03,1.087155
2,ENSG00000079974,ENSG00000079974,tss_distance,sum,0.272661,-8.895558e-07,8.109324e-07,1.203309


### Combine p-values per gene across anntoations using ACAT

We use the ACAT test to combine p-values across different annotations for each gene. This approach provides a single p-value per gene, accounting for multiple annotations.

In [34]:
combined = rdf.dropna(subset=["pv"]).groupby("egene")["pv"].agg(lambda x: acat_test(x.values)).reset_index()
combined.sort_values("pv")

,egene,pv
141,ENSG00000100376,0.000088
88,ENSG00000100219,0.000254
215,ENSG00000183172,0.001849
78,ENSG00000100154,0.005039
287,ENSG00000212939,0.005233
...,...,...
110,ENSG00000100294,0.984078
216,ENSG00000183473,0.985963
251,ENSG00000185838,0.986744
307,ENSG00000233903,0.988543


## SKAT tests
At the moment only default weighting with weights = Beta(MAF, 1, 25) is supported

In [ ]:
results = []

for gene, row in tqdm(dd.C.var.iterrows(), total=dd.shape[3]):
    Y = gaussianize(dd.G.obsm[pb_gex_key][[gene]].values + 1e-5 * np.random.randn(dd.shape[0], 1))

    start = max(0, row.start - cis_window)
    end = row.end + cis_window
    _G = dd.G[:, (dd.G.var.pos < end)]
    _G = _G[:, (_G.var.pos > start)]
    _G = _G[:, (_G.X.std(0) != 0)]

    rdict = run_skat_test(_G, Y, F, gene)
    results.append(rdict)

rdf = pd.DataFrame(results)
rdf

  0%|          | 0/360 [00:00<?, ?it/s]

[2025-04-22 16:06:29,662] INFO:root: Starting optimization ...
[2025-04-22 16:06:29,691] INFO:root: Starting optimization ...


[2025-04-22 16:06:29,741] INFO:root: Starting optimization ...


  1%|          | 3/360 [00:00<00:13, 26.89it/s]

[2025-04-22 16:06:29,775] INFO:root: Starting optimization ...
[2025-04-22 16:06:29,809] INFO:root: Starting optimization ...
[2025-04-22 16:06:29,845] INFO:root: Starting optimization ...


...

[2025-04-22 16:06:46,750] INFO:root: Starting optimization ...
[2025-04-22 16:06:46,794] INFO:root: Starting optimization ...
[2025-04-22 16:06:46,838] INFO:root: Starting optimization ...


 99%|█████████▉| 357/360 [00:17<00:00, 21.40it/s]

[2025-04-22 16:06:46,881] INFO:root: Starting optimization ...
[2025-04-22 16:06:46,919] INFO:root: Starting optimization ...
[2025-04-22 16:06:46,964] INFO:root: Starting optimization ...


100%|██████████| 360/360 [00:17<00:00, 20.77it/s]


,burden_gene,egene,weight_col,pv
0,ENSG00000100181,ENSG00000100181,maf_beta,0.024738
1,ENSG00000237438,ENSG00000237438,maf_beta,0.012391
2,ENSG00000177663,ENSG00000177663,maf_beta,0.526781
3,ENSG00000069998,ENSG00000069998,maf_beta,0.359992
4,ENSG00000185837,ENSG00000185837,maf_beta,0.693745
...,...,...,...,...
355,ENSG00000205560,ENSG00000205560,maf_beta,0.820957
356,ENSG00000100288,ENSG00000100288,maf_beta,0.433113
357,ENSG00000205559,ENSG00000205559,maf_beta,0.205992
358,ENSG00000100299,ENSG00000100299,maf_beta,0.350120
